In [1]:
%matplotlib inline
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils

In [2]:
from keras.models import Model, load_model
from keras.layers import Input, Embedding, LSTM, Dense, Dropout, Flatten
from keras.models import Model

Using TensorFlow backend.


In [3]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/encoder-decoder/utils.py'>

In [4]:
word2ind, ind2word = utils.load_index_word_map()

In [5]:
num_encoder_tokens = len(word2ind)
num_decoder_tokens = len(word2ind)
emb_dim = 50
latent_dim = 100

In [6]:
num_encoder_tokens

76

In [6]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, emb_dim)
encoder_embedding_outputs = encoder_embedding(encoder_inputs)
encoder = LSTM(latent_dim, return_state=True, dropout=0.2, recurrent_dropout=0.2)
encoder_outputs, state_h, state_c = encoder(encoder_embedding_outputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_embedding = Embedding(num_decoder_tokens, emb_dim)
decoder_embedding_outputs = decoder_embedding(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding_outputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [7]:
model.load_weights('model_weights.h5')

In [8]:
# Run training
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [9]:
encoder_input_data = pd.read_csv('encoder_input.csv').values
decoder_input_data = pd.read_csv('decoder_input.csv').values
decoder_target_data_2d = pd.read_csv('decoder_target.csv').values
# decoder_target_data = decoder_target_data.reshape((decoder_target_data.shape[0], decoder_target_data.shape[1], -1))

In [10]:
decoder_target_data = decoder_target_data_2d.reshape((*decoder_target_data_2d.shape, -1))

In [11]:
def train_valid_split(length, test_size=0.3, random_state=43):
    np.random.RandomState(seed=random_state)
    choices = list(range(length))
    val_choices = np.random.choice(
        choices,
        int(length*test_size),
        replace=False
    ).tolist()
    train_choices = list(set(choices) - set(val_choices))
    return train_choices, val_choices

In [12]:
encoder_input_data.shape

(1898365, 100)

In [13]:
decoder_input_data.shape

(1898365, 100)

In [14]:
decoder_target_data.shape

(1898365, 100, 1)

In [15]:
train_set, val_set = train_valid_split(encoder_input_data.shape[0])

In [16]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None)          0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, None)          0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, None, 50)      3800        input_1[0][0]                    
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, None, 50)      3800        input_2[0][0]                    
___________________________________________________________________________________________

In [17]:
model.fit([encoder_input_data[train_set], decoder_input_data[train_set]], decoder_target_data[train_set],
          batch_size=1000,
          epochs=20,
          validation_split=0.2)

Train on 1063084 samples, validate on 265772 samples
Epoch 1/20
1063084/1063084 [==============================] - 234s - loss: 1.4010 - val_loss: 1.3433
Epoch 2/20
1063084/1063084 [==============================] - 231s - loss: 1.3989 - val_loss: 1.3424
Epoch 3/20
1063084/1063084 [==============================] - 231s - loss: 1.3971 - val_loss: 1.3410
Epoch 4/20
1063084/1063084 [==============================] - 233s - loss: 1.3954 - val_loss: 1.3393
Epoch 5/20
1063084/1063084 [==============================] - 231s - loss: 1.3937 - val_loss: 1.3382
Epoch 6/20
1063084/1063084 [==============================] - 231s - loss: 1.3921 - val_loss: 1.3371
Epoch 7/20
1063084/1063084 [==============================] - 231s - loss: 1.3906 - val_loss: 1.3367
Epoch 8/20
1063084/1063084 [==============================] - 231s - loss: 1.3892 - val_loss: 1.3358
Epoch 9/20
1063084/1063084 [==============================] - 231s - loss: 1.3880 - val_loss: 1.3345
Epoch 10/20
1063084/1063084 [=========

In [18]:
model.save_weights('model_weights.h5')

In [19]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, emb_dim)
encoder_embedding_outputs = encoder_embedding(encoder_inputs)
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embedding_outputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_embedding = Embedding(num_decoder_tokens, emb_dim)
decoder_embedding_outputs = decoder_embedding(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding_outputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.load_weights('model_weights.h5')

encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding_outputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [20]:
encoder_model.save('encoder_model.h5')
decoder_model.save('decoder_model.h5')

In [23]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.array(word2ind[' ']).reshape((1, 1))

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.random.choice(list(range(num_decoder_tokens)), p=output_tokens[0, -1, :])
        sampled_char = ind2word[str(sampled_token_index)]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if len(decoded_sentence) > 200:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.array(sampled_token_index).reshape((1, 1))

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [24]:
print(decode_sequence(encoder_input_data[0]))

it, lay know this love up,  
Me not a reaser, now failingra's the weaking to never gazed what you need your arms, (in the christmas goar 'say)  
  
And I'll really know that you can't ask black through
